# Relationships between Variables

In [ ]:
import sys
sys.path.append('lib')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import brfss
import nsfg
from cdf import Cdf

In [ ]:
# plotting
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()

In [ ]:
# worksheet
from IPython.core.pylabtools import figsize
figsize(9, 6)

## Scatter plots

I'll start with the data from the BRFSS again.

In [ ]:
df = brfss.read_brfss().dropna(subset=['height', 'weight'])

I'll extract the height in cm and the weight in kg of the respondents in the sample.

In [ ]:
sample = df.sample(n=5000, replace=False)

In [ ]:
heights, weights = sample.height, sample.weight

Here's a simple scatter plot with `alpha=1`, so each data point is fully saturated.

In [ ]:
p = sns.scatterplot(
    data = sample,
    x = 'height',
    y = 'weight',
    alpha = 1
)
p.set(
    xlabel = 'Height (cm)',
    ylabel = 'Weight (kg)'
);

The data fall in obvious columns because they were rounded off.  We can reduce this visual artifact by adding some random noice to the data.

NOTE: The version of `Jitter` in the book uses noise with a uniform distribution.  Here I am using a normal distribution.  The normal distribution does a better job of blurring artifacts, but the uniform distribution might be more true to the data.

In [ ]:
def jitter(values: np.array, jitter=0.5) -> np.array:
    return np.random.normal(0, jitter, len(values)) + values

Heights were probably rounded off to the nearest inch, which is 2.8 cm, so I'll add random values from -1.4 to 1.4.

In [ ]:
# jitter is functional in sns at the moment
p = sns.scatterplot(
    x = jitter(heights, 1.4),
    y = jitter(weights, 0.5)
)
p.set(
    xlabel = 'Height (cm)',
    ylabel = 'Weight (kg)'
);

And here's what the jittered data look like.

The columns are gone, but now we have a different problem: saturation.  Where there are many overlapping points, the plot is not as dark as it should be, which means that the outliers are darker than they should be, which gives the impression that the data are more scattered than they actually are.

This is a surprisingly common problem, even in papers published in peer-reviewed journals.

We can usually solve the saturation problem by adjusting `alpha` and the size of the markers, `s`.

In [ ]:
xlim = (140, 210,)
ylim = (20, 200,)

In [ ]:
p = sns.scatterplot(
    x = jitter(heights, 1.4),
    y = jitter(weights, 0.5),
    color = 'darkblue',
    alpha = 0.2,
    s=10
)
p.set(
    xlabel = 'Height (cm)',
    ylabel = 'Weight (kg)',
    ylim = ylim,
    xlim = xlim
);

That's better.  This version of the figure shows the location and shape of the distribution most accurately.  There are still some apparent columns and rows where, most likely, people reported their height and weight using rounded values.  If that effect is important, this figure makes it apparent; if it is not important, we could use more aggressive jittering to minimize it. 

An alternative to a scatter plot is something like a `HexBin` plot, which breaks the plane into bins, counts the number of respondents in each bin, and colors each bin in proportion to its count.

In [ ]:
import matplotlib
cmap=matplotlib.cm.Blues

In [ ]:
plt.hexbin(
    heights.values,
    weights.values,
    cmap=matplotlib.cm.Blues
);
plt.xlabel('Height (cm)');
plt.ylabel('Weight (kg)');
# plt.xlim(xlim);
# plt.ylim(ylim);

In this case the binned plot does a pretty good job of showing the location and shape of the distribution.  It obscures the row and column effects, which may or may not be a good thing.

**Exercise:**  So far we have been working with a subset of only 5000 respondents.  When we include the entire dataset, making an effective scatterplot can be tricky.  As an exercise, experiment with `Scatter` and `HexBin` to make a plot that represents the entire dataset well.

In [ ]:
# With smaller markers, I needed more aggressive jittering to
# blur the measurement artifacts

# With this dataset, using all of the rows might be more trouble
# than it's worth.  Visualizing a subset of the data might be
# more practical and more effective.

plt.scatter(
    jitter(df.height.values, 2.8),
    jitter(df.weight.values, 1.8),
    alpha=0.01,
    s=2
);
plt.xlabel('Height (cm)');
plt.ylabel('Weight (kg)');
plt.xlim(xlim);
plt.ylim(ylim);

## Plotting percentiles

Sometimes a better way to get a sense of the relationship between variables is to divide the dataset into groups using one variable, and then plot percentiles of the other variable.

Divide the dataset into groups by height.

In [ ]:
bins = np.arange(135, 210, 5)
indices = np.digitize(df.height, bins)
groups = df.groupby(indices)

Here are the number of respondents in each group:

In [ ]:
for i, group in groups:
    print(f'{i+1:2}. {len(group)}')

Now we can compute the CDF of weight within each group.

In [ ]:
mean_heights = [group.height.mean() for _, group in groups]

In [ ]:
cdfs = [Cdf.from_seq(group.weight) for _, group in groups]

And then extract the 25th, 50th, and 75th percentile from each group.

In [ ]:
for percent in [75, 50, 25]:
    weight_percentiles = [cdf.percentile(percent) for cdf in cdfs]
    plt.plot(
        mean_heights,
        weight_percentiles,
        label = f'{percent:d}th'
    )
plt.xlabel('Height (cm)');
plt.ylabel('Weight (kg)');
plt.xlim(xlim);
plt.ylim(ylim);
plt.legend(loc='upper right');

**Exercise:** Yet another option is to divide the dataset into groups and then plot the CDF for each group.  As an exercise, divide the dataset into a smaller number of groups and plot the CDF for each group.

In [ ]:
bins = np.arange(140, 210, 10)
indices = np.digitize(df.height, bins)
groups = df.groupby(indices)
cdfs = [Cdf.from_seq(group.weight) for _, group in groups]

In [ ]:
for cdf in cdfs:
    plt.plot(
        cdf.xs,
        cdf.ps
    )
plt.ylabel('CDF');
plt.xlim(ylim);
plt.xlabel('Weight (kg)');

## Correlation

The following function computes the covariance of two variables using NumPy's `dot` function.

In [ ]:
def cov(xs: np.array, ys: np.array, meanx=None, meany=None):
    if meanx is None:
        meanx = np.mean(xs)
    if meany is None:
        meany = np.mean(ys)
    cov = np.dot(xs-meanx, ys-meany) / len(xs)
    return cov

And here's an example:

In [ ]:
cov(df.height, df.weight)

Covariance is useful for some calculations, but it doesn't mean much by itself.  The coefficient of correlation is a standardized version of covariance that is easier to interpret.

In [ ]:
def corr(xs: np.array, ys: np.array):
    return cov(xs, ys) / np.sqrt(xs.var() * ys.var())

The correlation of height and weight is about 0.51, which is a moderately strong correlation.

In [ ]:
corr(heights, weights)

NumPy provides a function that computes correlations, too:

In [ ]:
heights.isna().sum()

In [ ]:
np.corrcoef(heights.astype(np.float64), weights.astype(np.float64))

The result is a matrix with self-correlations on the diagonal (which are always 1), and cross-correlations on the off-diagonals (which are always symmetric).

Pearson's correlation is not robust in the presence of outliers, and it tends to underestimate the strength of non-linear relationships.

Spearman's correlation is more robust, and it can handle non-linear relationships as long as they are monotonic.  Here's a function that computes Spearman's correlation:

In [ ]:
def spearman_corr(xs, ys):
    xranks = pd.Series(xs).rank()
    yranks = pd.Series(ys).rank()
    return corr(xranks, yranks)

For heights and weights, Spearman's correlation is a little higher:

In [ ]:
spearman_corr(heights, weights)

A Pandas `Series` provides a method that computes correlations, and it offers `spearman` as one of the options.

In [ ]:
def spearman_corr(xs, ys):
    xs = pd.Series(xs)
    ys = pd.Series(ys)
    return xs.corr(ys, method='spearman')

The result is the same as for the one we wrote.

In [ ]:
spearman_corr(heights, weights)

In [ ]:
heights.corr(weights, method='spearman')

An alternative to Spearman's correlation is to transform one or both of the variables in a way that makes the relationship closer to linear, and the compute Pearson's correlation.

In [ ]:
corr(heights, np.log(weights))

## Exercises

Using data from the NSFG, make a scatter plot of birth weight versus mother’s age. Plot percentiles of birth weight versus mother’s age. Compute Pearson’s and Spearman’s correlations. How would you characterize the relationship between these variables?

In [ ]:
df = nsfg.read_live_fem_preg().dropna(subset=['agepreg', 'totalwgt_lb'])

In [ ]:
for method in('pearson', 'spearman'):
    print(f'{method}: {df.agepreg.corr(df.totalwgt_lb, method=method)}')

In [ ]:
# Solution

ages = live.agepreg
weights = live.totalwgt_lb
print('Corr', Corr(ages, weights))
print('SpearmanCorr', SpearmanCorr(ages, weights))

In [ ]:
bins = np.arange(10, 48, 3)
indices = np.digitize(df.agepreg, bins)
groups = df.groupby(indices)
# remove first and last groups
ages = [group.agepreg.mean() for i, group in groups][1:-1]
cdfs = [Cdf.from_seq(group.totalwgt_lb) for i, group in groups][1:-1]

In [ ]:
for percent in [75, 50, 25]:
    weights = [cdf.percentile(percent) for cdf in cdfs]
    label = '%dth' % percent
    plt.plot(ages, weights, label=f'{percent:d}th')
plt.xlabel('Mother\'s age (years)')
plt.ylabel('Birth weight (lbs)')
plt.xlim([14, 45])
plt.legend(loc='upper left');

In [ ]:
p = sns.scatterplot(
    data=df,
    x='agepreg',
    y='totalwgt_lb',
    alpha=0.05,
    color='darkblue',
    s=10
);
p.set(
    xlabel = 'Mother\'s age (years)',
    ylabel = 'Birth weight (lbs)',
    ylim = [0, 15],
    xlim = [10, 45]
);

1. The scatterplot shows a weak relationship between the variables but it is hard to see clearly.
2. The correlations support this.  Pearson's is around 0.07, Spearman's is around 0.09.  The difference between them suggests some influence of outliers or a non-linear relationsip.
3. Plotting percentiles of weight versus age suggests that the relationship is non-linear.  Birth weight increases more quickly in the range of mother's age from 15 to 25.  After that, the effect is weaker.